In [2]:
import os
import pandas as pd
import glob
import json
import re
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

## Load Completions

In [3]:
def load_all_jsonl_to_dataframe(folder_path):
    """Load all JSONL files from a folder into a pandas DataFrame."""
    # Find all .jsonl files
    jsonl_files = glob.glob(os.path.join(folder_path, "*.jsonl"))
    
    # Create list to hold dataframes
    dfs = []
    
    for file_path in jsonl_files:
        try:
            # Read file into dataframe
            df = pd.read_json(file_path, lines=True)
            df['source_file'] = os.path.basename(file_path)  # Optional: track source file
            dfs.append(df)
            print(f"Loaded {len(df)} rows from {os.path.basename(file_path)}")
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
    
    # Combine all dataframes
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        print(f"Created DataFrame with {len(combined_df)} rows from {len(jsonl_files)} files")
        return combined_df
    else:
        print("No valid files found")
        return pd.DataFrame()

# Usage
folder_path = "../data/deepscaler_aime_sample_eval_check/"
combined_data = load_all_jsonl_to_dataframe(folder_path)

Loaded 4 rows from 0.jsonl
Loaded 4 rows from 1.jsonl
Created DataFrame with 8 rows from 2 files


## Display sample data

In [4]:
# Display the first few rows
# combined_data.head(10)

In [5]:
# combined_data.iloc[0]

## Solution Extraction Utilities

Functions to extract solutions from model outputs and compute correctness.

In [6]:
def remove_boxed(s):
    if s is None:
        return None

    left = "\\boxed{"

    if s[:len(left)] != left:
        return None
    if s[-1] != "}":
        return None

    return s[len(left):-1]


def last_boxed_only_string(string):
    idx = string.rfind("\\boxed")
    if "\\boxed " in string:
        return "\\boxed " + string.split("\\boxed ")[-1].split("$")[0]
    if idx < 0:
        idx = string.rfind("\\fbox")
        if idx < 0:
            return None

    i = idx
    right_brace_idx = None
    num_left_braces_open = 0
    while i < len(string):
        if string[i] == "{":
            num_left_braces_open += 1
        if string[i] == "}":
            num_left_braces_open -= 1
            if num_left_braces_open == 0:
                right_brace_idx = i
                break
        i += 1

    if right_brace_idx is None:
        retval = None
    else:
        retval = string[idx:right_brace_idx + 1]

    return retval

def extract_solution(text):
    return remove_boxed(last_boxed_only_string(text))

def correctness_reward_func(response: str, actual_answers: str) -> float:
    extracted_answer = extract_solution(response)
    return 1.0 if extracted_answer == actual_answers else 0.0

## Transform for Analysis

In [7]:
check = combined_data.copy()[['prompt', 'answer']].drop_duplicates()
print(f"len(check): {len(check)}")

len(check): 8


In [8]:
df = combined_data.copy()
df['question_text'] = df['prompt']
df['ground_truth'] = df['answer']
df.drop([
    'source_file',
], axis=1, inplace=True)
# print(f"len(df): {len(df)}")
df = df.explode('outputs')
df['response'] = df['outputs'].apply(lambda x: x['output'])
# print(f"len(df): {len(df)}")
df['extracted_solution'] = df['response'].apply(lambda x: extract_solution(x))

# # # Only keep completions where there was a generated solution.
# # df = df[df['extracted_solution'].notna()]

df['reward'] = (df['extracted_solution'] == df['ground_truth'].astype(str)).astype(float)
# df
# df[df['extracted_solution'].isna()]

## Spot Check Traces

In [9]:
prompt_id = 1
correct_pos = 0
incorrect_pos = 0

prompt = df[df.prompt_id == prompt_id]
prompt_correct = prompt[prompt.reward == 1.0]
prompt_incorrect = prompt[prompt.reward == .0]

In [10]:
print(prompt.iloc[correct_pos]['prompt'])

Let $A$, $B$, $C$, and $D$ be point on the hyperbola $\frac{x^2}{20}- \frac{y^2}{24} = 1$ such that $ABCD$ is a rhombus whose diagonals intersect at the origin. Find the greatest real number that is less than $BD^2$ for all such rhombi. Let's think step by step and output the final answer within \boxed{}. Let's think step by step and output the final answer within \boxed{}.


In [11]:
print(prompt_correct.iloc[incorrect_pos]['response'])

"
<think>
Okay, so I have this problem where points A, B, C, D are on the hyperbola given by the equation x²/20 - y²/24 = 1. These points form a rhombus ABCD, and the diagonals of the rhombus intersect at the origin. I need to find the greatest real number that is less than BD² for all such rhombi. Then, that number will be the answer.

First, let's recall some properties of a rhombus. A rhombus is a quadrilateral with all sides equal, and its diagonals bisect each other at right angles. But in this problem, the diagonals intersect at the origin, which means the origin is the midpoint of both diagonals.

Given that the hyperbola is x²/20 - y²/24 = 1, it's a standard hyperbola centered at the origin, opening left and right. The asymptotes would be y = ±(sqrt(24/20))x = ±(sqrt(6/5))x, but maybe that's not immediately relevant.

Now, since the diagonals intersect at the origin, and it's a rhombus, the diagonals must be perpendicular. So, if one diagonal is along the x-axis, the other woul

In [12]:
print(prompt_incorrect.iloc[1]['response'])

 Let's think step step step.

<think>
Okay, so I need to solve this problem about a rhombus ABCD on a hyperbola, with diagonals intersecting at the origin. The hyperbola is given by the equation x²/20 - y²/24 = 1. The task is to find the greatest real number that is less than BD² for all such rhombi, and that number is to be the infimum (greatest lower bound) of BD². So essentially, I need to find the minimum possible value of BD² for such rhombi and then find the greatest number less than that minimum.

First, let's recall some properties of a rhombus. In a rhombus, all sides are equal, and the diagonals bisect each other at right angles. But wait, in this problem, the diagonals intersect at the origin, which is the center of the hyperbola. So perhaps the diagonals are along the coordinate axes?

Wait, not necessarily. Wait, the hyperbola is centered at the origin, and the diagonals of the rhombus intersect at the origin. So in a rhombus, the diagonals bisect each other. So if the dia

## Measure Perf

In [8]:
def pass_at_k(n, c, k):
    """
    :param n: total number of samples :param c: number of correct
    Samples
    :param k: k in pass@$k$
    """
    if n - c < k: return 1.0
    return 1.0 - np.prod(1.0 - k / np.arange(n - c + 1, n + 1))

# pass_at_k(16, 2, 1)

In [9]:
perf = (
    df[['question_text', 'ground_truth', 'reward']]
    .groupby(['question_text', 'ground_truth'])
    .agg(
        count=('reward', 'count'),
        sum_reward=('reward', 'sum'),
        accuracy=('reward', 'mean')
    )
)

# print(f"len(perf): {len(perf)}")

# If only answer with an extarcted \boxed{...} answer are kept, count isn't always 16.
# assert(len(perf[perf['count'] != 16]) == 0)
# perf[f'pass@{k}'] = perf['sum_reward'].apply(lambda x: pass_at_k(16, x, k))

k = 1
perf[f'pass@{k}'] = perf.apply(
    lambda x: pass_at_k(x['count'], x['sum_reward'], k), 
    axis=1
)
# perf

In [10]:
tweak_dataset = [
    {
        'problem': r['question_text'],
        'answer': str(r['ground_truth']),
        f'pass@{k}': str(r[f'pass@{k}']),
    }
    for index, r in perf.reset_index().iterrows()
]
# tweak_dataset

In [11]:
pass_at_k = f'pass@{k}'
for p in tweak_dataset:
    print(f"problem {p['problem']}")
    print(f"answer {p['answer']}")
    print(f"{pass_at_k} {p[pass_at_k]}")
    print()

problem A list of positive integers has the following properties:
$\bullet$ The sum of the items in the list is $30$.
$\bullet$ The unique mode of the list is $9$.
$\bullet$ The median of the list is a positive integer that does not appear in the list itself.
 Find the sum of the squares of all the items in the list. Let's think step by step and output the final answer within \boxed{}. Let's think step by step and output the final answer within \boxed{}.
answer 236
pass@1 0.3749999999999998

problem Let $A$, $B$, $C$, and $D$ be point on the hyperbola $\frac{x^2}{20}- \frac{y^2}{24} = 1$ such that $ABCD$ is a rhombus whose diagonals intersect at the origin. Find the greatest real number that is less than $BD^2$ for all such rhombi. Let's think step by step and output the final answer within \boxed{}. Let's think step by step and output the final answer within \boxed{}.
answer 480
pass@1 0.1875

problem Let $ABC$ be a triangle inscribed in circle $\omega$. Let the tangents to $\omega$ a